#

# INTEGRANTES

* SANCHEZ GUZMAN BRANDO CARLO



# CONEXIÓN A DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# INSTALACION DE LIBRERIAS

In [2]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rapidfuzz import process,fuzz

# DEFINICIÓN DE RUTAS



In [4]:
path_web_orders = '/content/drive/MyDrive/Colab Notebooks/MCD/GDD/Web_orders.txt'
path_catalog_orders = '/content/drive/MyDrive/Colab Notebooks/MCD/GDD/Catalog_Orders.txt'
path_products = '/content/drive/MyDrive/Colab Notebooks/MCD/GDD/products.txt'

# FUNCIONES

In [5]:
def resumen_estadistico(df, nombre_dataset):
    print(f"--- RESUMEN ESTADÍSTICO: {nombre_dataset} ---")

    print("\n1. INFORMACIÓN GENERAL Y TIPOS:")
    print(df.info())

    print("\n2. ESTADÍSTICAS NUMÉRICAS:")
    print(df.describe())

    print("\n3. VARIANZA (Numeric only):")
    print(df.select_dtypes(include=['number']).var())

    print("\n4. ESTADÍSTICAS CATEGÓRICAS (Unicidad y Frecuencia):")
    print(df.describe(include=['object']))

    print("\n5. VALORES ÚNICOS (Cardinalidad):")
    print(df.nunique())

    print("\n6. VALORES NULOS:")
    print(df.isnull().sum())
    print("-" * 50 + "\n")

## Correción de delimitador
DATASET: web_orders

Hay 2 opciones:


*   Cambiar el delimitador del encabezado de "," a ";"
*   Cambiar todos los delimitadores de ";" a "," y asegurarse que la columna que contiene "," dentro de sus registros esten entre comillas doble

Se opta por la opción 1, ya que involucra menos pasos y menos iteración de las filas.

In [6]:
with open(path_web_orders, 'r', encoding='utf-8') as f:
    lineas = f.readlines()

In [7]:
lineas[0] = lineas[0].replace(',', ';')

In [8]:
path_new_web_orders = 'Web_orders_csv.txt'
with open(path_new_web_orders, 'w', encoding='utf-8') as f:
    f.writelines(lineas)

# VALIDACIÓN DE FORMATO

## Verificación de formato
Se verifica archivo por archivo, si la cantidad de comillas dobles son par, sino, se debe corregir.

In [9]:
paths = [path_new_web_orders,path_catalog_orders,path_products]

In [10]:
for path in paths:
    print(f"\nVerificando archivo: {path}")
    try:
        with open(path, 'r', encoding='utf-8') as f:
            for i, linea in enumerate(f, start=1):
                num_comillas = linea.count('"')
                if num_comillas % 2 != 0:
                    print(f"Línea {i} con número impar de comillas ({num_comillas}):")
                    print(f"    {linea.strip()}")
    except Exception as e:
        print(f"Error leyendo {path}: {e}")


Verificando archivo: Web_orders_csv.txt

Verificando archivo: /content/drive/MyDrive/Colab Notebooks/MCD/GDD/Catalog_Orders.txt
Línea 125 con número impar de comillas (5):
    124,139856.00,8/97/29 00:00:00," ,"TY4400",1.00,"1353"

Verificando archivo: /content/drive/MyDrive/Colab Notebooks/MCD/GDD/products.txt


## Correción de formato (comilla faltante)
DATASET: catalog_orders

ID: 124

In [11]:
path_new_catalog_orders = '/content/drive/MyDrive/Colab Notebooks/MCD/GDD/catalog_orders_csv.txt'
with open(path_catalog_orders, 'r', encoding='utf-8') as f:
    lineas = f.readlines()
lineas[124] = '124,139856.00,8/97/29 00:00:00,"","TY4400",1.00,"1353"\n'
with open(path_new_catalog_orders, 'w', encoding='utf-8') as f:
    f.writelines(lineas)

## Importación de dataset y definición de delimitadores
Como el dataset web_orders le cambiamos el delimitador, lo definimos en esta línea de código.



In [12]:
df_web_orders = pd.read_csv(path_new_web_orders, sep=';')
df_catalog_orders = pd.read_csv(path_new_catalog_orders, sep=',')
df_products = pd.read_csv(path_products, sep=',')

# RESUMEN ESTADÏSTICO

In [13]:
resumen_estadistico(df_products, "Products")
resumen_estadistico(df_catalog_orders, "Catalog Orders")
resumen_estadistico(df_web_orders, "Web Orders")

--- RESUMEN ESTADÍSTICO: Products ---

1. INFORMACIÓN GENERAL Y TIPOS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        192 non-null    int64  
 1   TYPE      192 non-null    object 
 2   DESCRIP   192 non-null    object 
 3   PRICE     192 non-null    float64
 4   COST      192 non-null    float64
 5   PCODE     192 non-null    object 
 6   supplier  192 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 10.6+ KB
None

2. ESTADÍSTICAS NUMÉRICAS:
               ID       PRICE        COST
count  192.000000  192.000000  192.000000
mean    96.500000   84.426771   55.593854
std     55.569776  117.442675   79.690758
min      1.000000    3.000000    1.000000
25%     48.750000   16.000000    9.227500
50%     96.500000   40.995000   22.150000
75%    144.250000   91.275000   63.892500
max    192.000000  600.000000  450.000000


# ESTANDARIZACIÓN - web_orders

## Estandarización de columnas

*   Las columnas DATE, PCODE, CATALOG no estan en su columna correspondiente.
*   La columna custnum esta en minusculas y el contenido de la fila no esta acorde con el contenido de sus registros.

In [14]:
df_web_orders.rename(columns = {'DATE':'PCODE','CATALOG':'DATE','PCODE':'CATALOG','custnum':'CUSTNAME'},inplace=True)

In [15]:
df_web_orders.head()

,ID,INV,PCODE,DATE,CATALOG,QTY,CUSTNAME
0,1,2513000.0,GD3200,17/12/2000 00:00:00,Gardening,3.0,"Rifkin, Mr. Bob"
1,2,2513000.0,PT1400,6/1/2001 00:00:00,Pets,1.0,"Rifkin, Mr. Bob"
2,3,2513000.0,PT2000,6/1/2001 00:00:00,Pet,1.0,"Rifkin, Mr. Bob"
3,4,2513000.0,PT2100,6/1/2001 00:00:00,Pets,1.0,"Rifkin, Mr. Bob"
4,5,2513000.0,PT2OOO,27/7/2000 00:00:00,Pets,4.0,"Rifkin, Mr. Bob"


In [16]:
df_web_orders.shape

(943, 7)

## Conversión de tipo de dato

### Analisis

In [17]:
df_web_orders.dtypes

,0
ID,int64
INV,float64
PCODE,object
DATE,object
CATALOG,object
QTY,float64
CUSTNAME,object


In [18]:
df_web_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        943 non-null    int64  
 1   INV       943 non-null    float64
 2   PCODE     943 non-null    object 
 3   DATE      943 non-null    object 
 4   CATALOG   943 non-null    object 
 5   QTY       940 non-null    float64
 6   CUSTNAME  943 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 51.7+ KB


### Conversión

In [19]:
df_web_orders['INV']= df_web_orders['INV'].astype('Int64')
df_web_orders['QTY']= df_web_orders['QTY'].astype('Int64')
df_web_orders['PCODE']= df_web_orders['PCODE'].astype('string')
df_web_orders['CATALOG']= df_web_orders['CATALOG'].astype('string')
df_web_orders['CUSTNAME']= df_web_orders['CUSTNAME'].astype('string')
df_web_orders['DATE'] = pd.to_datetime(df_web_orders['DATE'], errors='coerce')

/tmp/ipython-input-1229009830.py:6: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_web_orders['DATE'] = pd.to_datetime(df_web_orders['DATE'], errors='coerce')


In [20]:
df_web_orders.head()

,ID,INV,PCODE,DATE,CATALOG,QTY,CUSTNAME
0,1,2513000,GD3200,2000-12-17,Gardening,3,"Rifkin, Mr. Bob"
1,2,2513000,PT1400,2001-01-06,Pets,1,"Rifkin, Mr. Bob"
2,3,2513000,PT2000,2001-01-06,Pet,1,"Rifkin, Mr. Bob"
3,4,2513000,PT2100,2001-01-06,Pets,1,"Rifkin, Mr. Bob"
4,5,2513000,PT2OOO,2000-07-27,Pets,4,"Rifkin, Mr. Bob"


### Verificación

In [21]:
df_web_orders.dtypes

,0
ID,int64
INV,Int64
PCODE,string[python]
DATE,datetime64[ns]
CATALOG,string[python]
QTY,Int64
CUSTNAME,string[python]


# SANITIZACIÓN - web_orders

## Verificación de nulos

In [22]:
df_web_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID        943 non-null    int64         
 1   INV       943 non-null    Int64         
 2   PCODE     943 non-null    string        
 3   DATE      943 non-null    datetime64[ns]
 4   CATALOG   943 non-null    string        
 5   QTY       940 non-null    Int64         
 6   CUSTNAME  943 non-null    string        
dtypes: Int64(2), datetime64[ns](1), int64(1), string(3)
memory usage: 53.5 KB


In [23]:
df_web_orders.isna().sum()

,0
ID,0
INV,0
PCODE,0
DATE,0
CATALOG,0
QTY,3
CUSTNAME,0


Se encuentran valores nulos en los ID: 628, 778, 785

In [24]:
df_web_orders[df_web_orders['QTY'].isna()]

,ID,INV,PCODE,DATE,CATALOG,QTY,CUSTNAME
627,628,9645000,PT3500,2001-01-10,Pets,<NA>,"King, Ms. Kathryn"
777,778,11650000,SP2400,2000-06-20,Sports,<NA>,"Murphy, Ms. Wendy"
784,785,11775000,GD2100,2001-02-22,Gardening,<NA>,"Jones, Ms. D. Flowers"


Al ser solo 3 valores, elegí eliminar los nulos.

In [25]:
df_web_orders = df_web_orders.dropna()

## Normalización de datos

### Normalización de PCODE
Hay 119 valores distintos de PCODE, pero no todos son correctos porque hay faltas ortograficas y errores de tipeo.

In [26]:
df_web_orders['PCODE'].value_counts()

,count
PCODE,
PT1400,24
SP2600,23
SP1000,23
SP3300,22
SP2400,22
...,...
PT3400,1
PT1OOO,1
sp2000,1


Se usará el dataset de products para poder estandarizar los valores de PCODE

In [27]:
df_products.head()

,ID,TYPE,DESCRIP,PRICE,COST,PCODE,supplier
0,1,Database,Track-It-All,560.00,425.0,SW1005,"Software America, Inc."
1,2,Database,Omnibus,499.95,450.0,SW1207,Software America
2,3,Database,Balboa,499.00,300.0,SW3049,Software America
3,4,Financial,Stock Market Analyst,69.95,45.0,SW2842,Software America
4,5,Financial,Tax Relief (Version 10.40),77.95,54.5,SW4187,Software America


Verificación de valores faltantes

In [28]:
valores_faltantes = df_web_orders.loc[~df_web_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

<StringArray>
['PT2OOO', 'GD16OO', 'TY21OO', 'SP14OO', 'TY1900', 'GD110O', 'gd3100',
 'TY17OO', 'GD15OO', 'SP13OO', 'TY16OO', 'GD12OO', 'GD101O', 'GD10)!',
 'GD12))', 'GD13OO', 'PT1OOO', 'sp2000', 'gd3200', 'GD11OO']
Length: 20, dtype: string


Se pondrá todo en mayuscula y se reemplaza la O por 0

In [29]:
df_web_orders['PCODE'] = df_web_orders['PCODE'].str.upper()
df_web_orders['PCODE'] = df_web_orders['PCODE'].str.replace('O', '0', regex=False)

In [30]:
valores_faltantes = df_web_orders.loc[~df_web_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

<StringArray>
['TY1900', 'GD10)!', 'GD12))']
Length: 3, dtype: string


In [31]:
df_products['PCODE'].unique()

array(['SW1005', 'SW1207', 'SW3049', 'SW2842', 'SW4187', 'SW5901',
       'SW1991', 'SW2123', 'SW3625', 'SW3802', 'SW3895', 'SW4155',
       'SW5338', 'SW6269', 'SW6531', 'SW7935', 'SW9521', 'SW9604',
       'SW9782', 'SW1526', 'SW3545', 'SW3617', 'SW3909', 'SW4550',
       'SW4773', 'SW4792', 'SW7013', 'SW7221', 'SW4575', 'SW5493',
       'SW6132', 'SW6352', 'SW7383', 'SW7573', 'SW7823', 'SW8241',
       'SW9023', 'SW7384', 'SW7585', 'SW8376', 'SW8511', 'SW2782',
       'SW2927', 'SW4323', 'SW5655', 'SW6180', 'SW7385', 'SW8001',
       'SW4831', 'SW5952', 'SW7271', 'SW1612', 'SW1807', 'SW1828',
       'SW2147', 'SW2181', 'SW2288', 'SW3104', 'SW3482', 'SW4825',
       'GD1001', 'GD1010', 'GD1100', 'GD1200', 'GD1300', 'GD1400',
       'GD1500', 'GD1600', 'GD2000', 'GD3000', 'GD3100', 'GD3200',
       'GD3300', 'GD3400', 'GD2100', 'GD2200', 'GD2300', 'GD2400',
       'GD4000', 'GD4100', 'GD4200', 'GD4300', 'GD4400', 'SP1000',
       'SP1100', 'SP1200', 'SP1300', 'SP1400', 'SP1500', 'SP16

Se utiliza la libreria rapidfuzz para hacer una estimación de similitud de los datos faltantes

In [32]:
referencia = df_products['PCODE'].dropna().astype(str).unique()
for valor in valores_faltantes:
    resultado = process.extractOne(valor, referencia, score_cutoff=60)
    print(f"{valor} → {resultado}")


TY1900 → ('TY1100', 83.33333333333334, 152)
GD10)! → ('GD1001', 66.66666666666667, 60)
GD12)) → ('GD1200', 66.66666666666667, 63)


Reemplazo en función de los resultados anteriores

In [33]:
reemplazos = {
    'TY1900': 'TY1100',
    'GD10)!': 'GD1001',
    'GD12))': 'GD1200'
}

In [34]:
df_web_orders['PCODE'] = df_web_orders['PCODE'].replace(reemplazos)

Verificación final de la normalización de la columna PCODE

In [35]:
valores_faltantes = df_web_orders.loc[~df_web_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

<StringArray>
[]
Length: 0, dtype: string


### Normalización de CATALOG

In [36]:
df_web_orders['CATALOG'].value_counts()

,count
CATALOG,
Sports,337
Toys,234
Gardening,190
Pets,171
Toy,2
Sport,2
Pet,1
Tosy,1
Gardenings,1


Se identifican las categorias más frecuentes para reasignar los errores en esas categorias

In [37]:
conteo = df_web_orders['CATALOG'].value_counts()
catalogos_frecuentes = conteo[conteo > 50].index.tolist()
print(catalogos_frecuentes)

['Sports', 'Toys', 'Gardening', 'Pets']


Se utliza la similitud con la libreria rapidfuzz para estimar la similitud de datos, tomo una similitud superior al 60%

In [38]:
reemplazos = {}
for valor in df_web_orders['CATALOG'].unique():
    if valor in catalogos_frecuentes:
        continue
    mejor_match = process.extractOne(valor, catalogos_frecuentes, scorer=fuzz.token_sort_ratio)
    if mejor_match and mejor_match[1] >= 60:
        reemplazos[valor] = mejor_match[0]
print(reemplazos)

{'Pet': 'Pets', 'Tosy': 'Toys', 'Gardenings': 'Gardening', 'Sport': 'Sports', 'Toy': 'Toys', 'Garden': 'Gardening'}


In [39]:
df_web_orders['CATALOG'] = df_web_orders['CATALOG'].replace(reemplazos)
df_web_orders['CATALOG'].value_counts()


,count
CATALOG,
Sports,339
Toys,237
Gardening,192
Pets,172


### Normalización de CUSTNAME

In [40]:
df_web_orders['CUSTNAME'].value_counts()

,count
CUSTNAME,
"Jones, Mr. J. T.",21
"Jones, Mr. T. J.",20
"Jones, Mr. Everett",15
"McDonald, Mr. Jim",14
"Kraft, Ms. Rose",13
...,...
"Jones, Mr. Bill A.",1
"Courtney, Ms. L.",1
"Kurtz, Mr. Allen",1


In [41]:
conteo = df_web_orders['CUSTNAME'].value_counts()
clientes_frecuentes = conteo[conteo > 10].index.tolist()
print(clientes_frecuentes)

['Jones, Mr. J. T.', 'Jones, Mr. T. J.', 'Jones, Mr. Everett', 'McDonald, Mr. Jim', 'Kraft, Ms. Rose', 'Perlmann, Mr. T. C.', 'Chang, Ms. Kate', 'James, Mr. Rick', 'Bating, Ms. S. T.', 'Jones, Mr. Brain', 'Buscemi, Mr. Steve', 'Janus, Mr. David', 'Hinfelt, Mr. B. K.']


No se ve ningún patron o posibles nombres mal escritos

In [42]:
reemplazos = {}
for valor in df_web_orders['CUSTNAME'].unique():
    if valor in clientes_frecuentes:
        continue  # Si el valor lo encuentra, no se hace nada
    mejor_match = process.extractOne(valor, clientes_frecuentes, scorer=fuzz.token_sort_ratio)
    if mejor_match and mejor_match[1] >= 80:  # Umbral que hace que los que tengan 90% de similitud o más
        reemplazos[valor] = mejor_match[0]
print(reemplazos)

{'Jones, Mr. B. T.': 'Jones, Mr. J. T.', 'Jones, Mr. Bob': 'Jones, Mr. Brain', 'Jones, Mr. Bill': 'Jones, Mr. Brain', 'Jones, Mr. Brainard': 'Jones, Mr. Brain', 'Jones, Mr. F. J.': 'Jones, Mr. J. T.', 'Jones, Mr. D. T.': 'Jones, Mr. J. T.'}


# ESTANDARIZACIÓN - catalog_orders

### Descripción de dataset

In [43]:
df_catalog_orders.head()

,ID,INV,DATE,CATALOG,PCODE,QTY,custnum
0,1,107707.0,3/97/7 00:00:00,Sports,SP1000,1.00,1242
1,2,110633.0,3/97/25 00:00:00,Gardening,GD2200,1.00,1243
2,3,111155.0,3/97/28 00:00:00,Pets,PT1300,3.00,1244
3,4,126867.0,6/97/22 00:00:00,Toys,TY4100,1.00,1245
4,5,135872.0,8/97/8 00:00:00,Toys,TY1200,4.00,1246


In [44]:
df_catalog_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6767 entries, 0 to 6766
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       6767 non-null   int64  
 1   INV      6767 non-null   float64
 2   DATE     6767 non-null   object 
 3   CATALOG  6764 non-null   object 
 4   PCODE    6767 non-null   object 
 5   QTY      6761 non-null   object 
 6   custnum  6767 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 370.2+ KB


### Renombre de columna



In [45]:
df_catalog_orders.rename(columns = {'custnum':'CUSTNUM'},inplace=True)

### Estandarizacón de QTY

In [46]:
df_catalog_orders['QTY'].value_counts()

,count
QTY,
1.00,4734
3.00,746
4.00,670
2.00,610
,1


Busco los valores vacios y como aún esta considerado como String, cualquier cadena vacia.

In [47]:
df_catalog_orders[df_catalog_orders['QTY'].isna() | (df_catalog_orders['QTY'].astype(str).str.strip() == '')]

,ID,INV,DATE,CATALOG,PCODE,QTY,CUSTNUM
410,411,167813.0,5/01/8 00:00:00,Software,SW4792,,1611
1003,1004,189914.0,5/98/17 00:00:00,Gardening,GD3000,NaN,2145
1218,1219,136327.0,8/97/10 00:00:00,Toys,TY3700,NaN,2339
2886,2887,150958.0,10/97/26 00:00:00,Collectibles,CC1100,NaN,894
2900,2901,150958.0,2/01/2 00:00:00,Collectibles,CC1100,NaN,9054
5890,5891,150958.0,10/97/26 00:00:00,Collectibles,CC1100,NaN,11145000
5904,5905,150958.0,2/01/2 00:00:00,Collectibles,CC1100,NaN,11145000


Ya que la columna tiene valores en string como 1.0, 2.0, entre otros, primero debo convertirlo a decimal y luego recien a entero.

In [48]:
df_catalog_orders['QTY'] = pd.to_numeric(df_catalog_orders['QTY'], errors='coerce')
df_catalog_orders['QTY']= df_catalog_orders['QTY'].astype('Int64')

Uso la misma validación anterior, pero ahora la parte que deberia ser la que devuelva es la funcion de "isna"

In [49]:
df_catalog_orders[df_catalog_orders['QTY'].isna() | (df_catalog_orders['QTY'].astype(str).str.strip() == '')]

,ID,INV,DATE,CATALOG,PCODE,QTY,CUSTNUM
410,411,167813.0,5/01/8 00:00:00,Software,SW4792,<NA>,1611
1003,1004,189914.0,5/98/17 00:00:00,Gardening,GD3000,<NA>,2145
1218,1219,136327.0,8/97/10 00:00:00,Toys,TY3700,<NA>,2339
2886,2887,150958.0,10/97/26 00:00:00,Collectibles,CC1100,<NA>,894
2900,2901,150958.0,2/01/2 00:00:00,Collectibles,CC1100,<NA>,9054
5890,5891,150958.0,10/97/26 00:00:00,Collectibles,CC1100,<NA>,11145000
5904,5905,150958.0,2/01/2 00:00:00,Collectibles,CC1100,<NA>,11145000


In [50]:
df_catalog_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6767 entries, 0 to 6766
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       6767 non-null   int64  
 1   INV      6767 non-null   float64
 2   DATE     6767 non-null   object 
 3   CATALOG  6764 non-null   object 
 4   PCODE    6767 non-null   object 
 5   QTY      6760 non-null   Int64  
 6   CUSTNUM  6767 non-null   int64  
dtypes: Int64(1), float64(1), int64(2), object(3)
memory usage: 376.8+ KB


### Conversión de tipo de dato

In [51]:
df_catalog_orders['INV']= df_catalog_orders['INV'].astype('Int64')
df_catalog_orders['PCODE']= df_catalog_orders['PCODE'].astype('string')
df_catalog_orders['CATALOG']= df_catalog_orders['CATALOG'].astype('string')
df_catalog_orders['CUSTNUM']= df_catalog_orders['CUSTNUM'].astype('Int64')
df_catalog_orders['DATE'] = pd.to_datetime(df_catalog_orders['DATE'],format='%m/%y/%d %H:%M:%S', errors='coerce')

### Estandarización CATALOG

#### Descripción de valores

In [52]:
df_catalog_orders['CATALOG'].value_counts()

,count
CATALOG,
Sports,1851
Pets,1671
Toys,1309
Gardening,933
Software,506
Collectibles,150
Sport,59
Toy,52
Softwares,31


Obtengo los catalogos más frecuentes, observando el conteo anterior, puedo poner un limite de mayor a 100

In [53]:
conteo = df_catalog_orders['CATALOG'].value_counts()
catalogos_frecuentes = conteo[conteo > 100].index.tolist()
print(catalogos_frecuentes)

['Sports', 'Pets', 'Toys', 'Gardening', 'Software', 'Collectibles']


#### Estimación por similitud

In [54]:
reemplazos = {}
for valor in df_catalog_orders['CATALOG'].dropna().unique():
    val_str = str(valor).strip()

    if val_str in catalogos_frecuentes:
        continue
    if val_str.isupper():
        matches = [c for c in catalogos_frecuentes if c.lower() == val_str.lower()]
        if matches:
            reemplazos[val_str] = matches[0]
            continue
    mejor_match = process.extractOne(val_str, catalogos_frecuentes, scorer=fuzz.token_sort_ratio)
    if mejor_match and mejor_match[1] >= 60:
        reemplazos[val_str] = mejor_match[0]
print(reemplazos)

{'Toy': 'Toys', 'Softwars': 'Software', 'Colectibles': 'Collectibles', 'Collectible': 'Collectibles', 'Pest': 'Pets', 'Sporst': 'Sports', 'Softwares': 'Software', 'Gardning': 'Gardening', 'Tots': 'Toys', 'Tosy': 'Toys', 'Sport': 'Sports', 'Pet': 'Pets', 'Softwar': 'Software', 'GARDENING': 'Gardening', 'Spots': 'Sports', 'Collectables': 'Collectibles', 'Prts': 'Pets', 'Pats': 'Pets'}


#### Verificación de catalog

In [55]:
df_catalog_orders['CATALOG'] = df_catalog_orders['CATALOG'].replace(reemplazos)
df_catalog_orders['CATALOG'].value_counts()

,count
CATALOG,
Sports,1952
Pets,1704
Toys,1392
Gardening,958
Software,577
Collectibles,181


#### Valores nulos de catalog

In [56]:
df_catalog_orders[df_catalog_orders['CATALOG'].isna() | (df_catalog_orders['CATALOG'].astype(str).str.strip() == '')]

,ID,INV,DATE,CATALOG,PCODE,QTY,CUSTNUM
123,124,139856,1997-08-29,<NA>,TY4400,1,1353
2680,2681,116156,2000-08-02,<NA>,CC3600,1,709
5684,5685,116156,2000-08-02,<NA>,CC3600,1,10201000


In [57]:
df_catalog_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6767 entries, 0 to 6766
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ID       6767 non-null   int64         
 1   INV      6767 non-null   Int64         
 2   DATE     6767 non-null   datetime64[ns]
 3   CATALOG  6764 non-null   string        
 4   PCODE    6767 non-null   string        
 5   QTY      6760 non-null   Int64         
 6   CUSTNUM  6767 non-null   Int64         
dtypes: Int64(3), datetime64[ns](1), int64(1), string(2)
memory usage: 390.0 KB


### Estandarización de PCODE

#### Descripción de PCODE

In [58]:
df_catalog_orders[df_catalog_orders.isna().any(axis=1)]

,ID,INV,DATE,CATALOG,PCODE,QTY,CUSTNUM
123,124,139856,1997-08-29,<NA>,TY4400,1,1353
410,411,167813,2001-05-08,Software,SW4792,<NA>,1611
1003,1004,189914,1998-05-17,Gardening,GD3000,<NA>,2145
1218,1219,136327,1997-08-10,Toys,TY3700,<NA>,2339
2680,2681,116156,2000-08-02,<NA>,CC3600,1,709
2886,2887,150958,1997-10-26,Collectibles,CC1100,<NA>,894
2900,2901,150958,2001-02-02,Collectibles,CC1100,<NA>,9054
5684,5685,116156,2000-08-02,<NA>,CC3600,1,10201000
5890,5891,150958,1997-10-26,Collectibles,CC1100,<NA>,11145000
5904,5905,150958,2001-02-02,Collectibles,CC1100,<NA>,11145000


In [59]:
df_catalog_orders['PCODE'].value_counts()

,count
PCODE,
SP2400,140
SP1000,129
SP2600,125
SP2200,118
SP2300,117
...,...
sw2181,1
sw2123,1
C3200,1


#### Valores faltantes

In [60]:
valores_faltantes = df_catalog_orders.loc[~df_catalog_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

<StringArray>
['sp2000', 'TY1900', 'TY17OO', 'pt3500', 'gd3100', 'pt4100',  'CC220',
 'SP14OO', 'SP13OO', 'TY16OO', 'PT2OOO', 'PT14OO', 'GD10!)', 'TY19OO',
 'gd2300', 'gd2200', 'GD16OO', 'cc1100', 'TY21OO', 'CC42OO', 'GD110O',
 'sw1991', 'gd3300', 'GD12OO', 'GD101O', 'gd3200', 'sw1612', 'sp2200',
 'GD10)!', 'sw1828', 'GD15OO', 'CC15OO', 'GD13OO', 'GD12))', 'sw2782',
 'CC43))', 'PT1OOO', 'sw2181', 'sw2123',  'C3200', 'GD11OO']
Length: 41, dtype: string


#### Correción de valores

In [61]:
df_catalog_orders['PCODE'] = df_catalog_orders['PCODE'].str.upper()
df_catalog_orders['PCODE'] = df_catalog_orders['PCODE'].str.replace('O', '0', regex=False)

#### Verificación posterior

In [62]:
valores_faltantes = df_catalog_orders.loc[~df_catalog_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

<StringArray>
['TY1900', 'CC220', 'GD10!)', 'GD10)!', 'GD12))', 'CC43))', 'C3200']
Length: 7, dtype: string


#### Estimación por similitud de valores faltantes y reemplazo

In [63]:
for valor in valores_faltantes:
    resultado = process.extractOne(valor, referencia, score_cutoff=60)
    print(f"{valor} : {resultado}")

TY1900 : ('TY1100', 83.33333333333334, 152)
CC220 : ('CC2200', 90.9090909090909, 131)
GD10!) : ('GD1001', 66.66666666666667, 60)
GD10)! : ('GD1001', 66.66666666666667, 60)
GD12)) : ('GD1200', 66.66666666666667, 63)
CC43)) : ('CC4300', 66.66666666666667, 145)
C3200 : ('CC3200', 90.9090909090909, 137)


In [64]:
reemplazos = {
    'TY1900': 'TY1100',
    'CC220': 'CC2200',
    'GD10!)': 'GD1010',
    'GD10)!': 'GD1001',
    'GD12))':'GD1200',
    'CC43))':'CC4300',
    'C3200':'CC3200'
}

In [65]:
df_catalog_orders['PCODE'] = df_catalog_orders['PCODE'].replace(reemplazos)

In [66]:
valores_faltantes = df_catalog_orders.loc[~df_catalog_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

<StringArray>
[]
Length: 0, dtype: string


### Imputación de datos de columna CATALOG mapeando la columna PCODE

In [67]:
df_catalog_orders[df_catalog_orders['CATALOG'].isna() | (df_catalog_orders['CATALOG'].astype(str).str.strip() == '')]

,ID,INV,DATE,CATALOG,PCODE,QTY,CUSTNUM
123,124,139856,1997-08-29,<NA>,TY4400,1,1353
2680,2681,116156,2000-08-02,<NA>,CC3600,1,709
5684,5685,116156,2000-08-02,<NA>,CC3600,1,10201000


#### Mapeo de valores de PCODE con su respectivo CATALOG

In [68]:
pcode_catalog_map = df_catalog_orders[df_catalog_orders['CATALOG'].notna()].drop_duplicates(subset=['PCODE']) \
                      .set_index('PCODE')['CATALOG']

In [69]:
print(pcode_catalog_map)

PCODE
SP1000       Sports
GD2200    Gardening
PT1300         Pets
TY4100         Toys
TY1200         Toys
            ...    
SW3049     Software
SW1526     Software
SW5655     Software
SW3104     Software
SW8001     Software
Name: CATALOG, Length: 184, dtype: string


In [70]:
catalog_faltantes = df_catalog_orders['CATALOG'].isna()
df_catalog_orders.loc[catalog_faltantes, 'CATALOG'] = df_catalog_orders.loc[catalog_faltantes, 'PCODE'].map(pcode_catalog_map)

#### Verificación de imputación

In [71]:
df_catalog_orders[df_catalog_orders['ID'].isin([124, 2681, 5685])]

,ID,INV,DATE,CATALOG,PCODE,QTY,CUSTNUM
123,124,139856,1997-08-29,Toys,TY4400,1,1353
2680,2681,116156,2000-08-02,Collectibles,CC3600,1,709
5684,5685,116156,2000-08-02,Collectibles,CC3600,1,10201000


### Estandarización de CUSTNUM

#### Descripción de CUSTNUM

Se encontro valores dentro de CUSTNUM que no son correctos ya que es en 3004 registros solo son una secuencia de numeros que no representan al código del cliente. Ya que un código de INV debe tener solo un CUSTNUM porque es quien ha comprado el producto

In [72]:
df_catalog_orders['CUSTNUM'].value_counts()

,count
CUSTNUM,
2349000,53
6944000,48
9770000,45
13405000,44
10770000,39
...,...
2205,1
2206,1
2207,1


In [73]:
custnum_counts = df_catalog_orders['CUSTNUM'].value_counts()
custnum_unicos = custnum_counts[custnum_counts == 1].index

In [74]:
map_data = df_catalog_orders[~df_catalog_orders['CUSTNUM'].isin(custnum_unicos)]

#### Mapeo de INV y su respectivo CUSTNUM

In [75]:
inv_to_cust_map = map_data.groupby('INV')['CUSTNUM'].agg(lambda x: x.value_counts().idxmax())
print(inv_to_cust_map)

INV
98689      8645000
98769      2349000
98849     11650000
98860     12900000
98894      3013000
            ...   
243353      137000
243374      387000
243375      262000
243376      387000
243377      137000
Name: CUSTNUM, Length: 1711, dtype: Int64


In [76]:
df_catalog_orders['CUSTNUM'].value_counts()

,count
CUSTNUM,
2349000,53
6944000,48
9770000,45
13405000,44
10770000,39
...,...
2205,1
2206,1
2207,1


Para cuestiones de seguridad, creo una columna adicional de CUSTNUM ORIGINAL y comprar los valores con el valore reemplazado, tambien esto me ayuda a saber cuantos valores han sido reemplazados

In [77]:
df_catalog_orders['CUSTNUM_ORIGINAL'] = df_catalog_orders['CUSTNUM']

In [78]:
custnum_a_reemplazar = df_catalog_orders['CUSTNUM'].isin(custnum_unicos)

In [79]:
df_catalog_orders.loc[custnum_a_reemplazar, 'CUSTNUM'] = df_catalog_orders.loc[custnum_a_reemplazar, 'INV'].map(inv_to_cust_map)

Se cambiaron 3004 registros, por el mapeo de INV con su CUSTNUM

In [80]:
cambios = df_catalog_orders[df_catalog_orders['CUSTNUM'] != df_catalog_orders['CUSTNUM_ORIGINAL']][['ID', 'INV', 'CUSTNUM_ORIGINAL', 'CUSTNUM']]
print(cambios)

        ID     INV  CUSTNUM_ORIGINAL   CUSTNUM
0        1  107707              1242    137000
1        2  110633              1243    137000
2        3  111155              1244    137000
3        4  126867              1245    137000
4        5  135872              1246    137000
...    ...     ...               ...       ...
2999  3000  235339               995  11525000
3000  3001  103305               996  11525000
3001  3002  110451               997  11525000
3002  3003  121407               998  11525000
3003  3004  125551               999  11525000

[3004 rows x 4 columns]


In [81]:
df_catalog_orders.shape

(6767, 8)

df_catalog_orders.info()

#### Eliminación de valores nulos

Aún tenemos nulos en la columna de QTY, es posible realizar la imputación de datos en función a la Media, o moda en función a su PCODE, pero no sería tan preciso, por lo que al ser solo 6 valores de un total de 6767, se eliminarán.
Tambien elimino la columna CUSTNUM_ORIGINAL que creé para verificación

In [82]:
df_catalog_orders = df_catalog_orders.dropna()
df_catalog_orders.drop(columns=['CUSTNUM_ORIGINAL'],inplace=True)

### Exportación de Dataset limpios

In [83]:
df_catalog_orders.to_csv('/content/drive/MyDrive/Colab Notebooks/MCD/GDD/catalog_orders_clean.csv', index=False, quoting=1)

In [84]:
df_web_orders.to_csv('/content/drive/MyDrive/Colab Notebooks/MCD/GDD/web_orders_clean.csv', index=False, quoting=1)

## ESTRATEGIA DE INTEGRACIÓN

### Eliminación los ID de los dataset ya limpios

In [85]:
df_web_orders = df_web_orders.drop(columns=['ID'], errors='ignore')
df_catalog_orders = df_catalog_orders.drop(columns=['ID'], errors='ignore')

### Agregado de columna ORIGEN

Se agrega una columna "ORIGEN" para poder rastrear y contabilizar el origen de los datos.

In [86]:
df_web_orders['ORIGEN'] = 'WEB'
df_catalog_orders['ORIGEN'] = 'CATALOG'

### Integrado de los dataset

In [87]:
df_orders = pd.concat([df_web_orders, df_catalog_orders], ignore_index=True)

### Creación de un ID generalizado para el dataset integrado

In [88]:
df_orders.insert(0, 'ID', range(1, len(df_orders) + 1))

### Verificación de nuevo dataset

In [89]:
df_orders['ORIGEN'].value_counts()

,count
ORIGEN,
CATALOG,6760
WEB,940


In [90]:
df_orders.head()

,ID,INV,PCODE,DATE,CATALOG,QTY,CUSTNAME,ORIGEN,CUSTNUM
0,1,2513000,GD3200,2000-12-17,Gardening,3,"Rifkin, Mr. Bob",WEB,<NA>
1,2,2513000,PT1400,2001-01-06,Pets,1,"Rifkin, Mr. Bob",WEB,<NA>
2,3,2513000,PT2000,2001-01-06,Pets,1,"Rifkin, Mr. Bob",WEB,<NA>
3,4,2513000,PT2100,2001-01-06,Pets,1,"Rifkin, Mr. Bob",WEB,<NA>
4,5,2513000,PT2000,2000-07-27,Pets,4,"Rifkin, Mr. Bob",WEB,<NA>


### Exportación de datos

Se exportan los dataset para luego importarlos al WEKA

In [91]:
df_orders.to_csv('/content/drive/MyDrive/Colab Notebooks/MCD/GDD/all_orders.csv', index=False, quoting=1)

In [92]:
df_products.to_csv('/content/drive/MyDrive/Colab Notebooks/MCD/GDD/products.csv', index=False, quoting=1)